In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
@author: M Arshad Zahangir Chowdhury

Analytics for IR dataset

"""

%matplotlib inline 

import sys
import os

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy import signal
from ipywidgets import interactive
import seaborn as sns  #heat map
import glob # batch processing of images

if '../../' not in sys.path:
    sys.path.append('../../')

from src.spectral_datasets.IR_datasets import IR_data

from src.misc.analytics import plot_compound_counts
from src.misc.analytics import plot_dataset_property
from src.misc.analytics import load_exp_spectra

## load IR spectra

In [ ]:
s = IR_data(data_start = 400, data_end = 4000, resolution=1, verbosity = True)

In [ ]:
s.load_IR_data()

## attributes of the IR dataset

In [ ]:
print('Number of Compounds:', s.n_compounds)
print('Number of Spectrum:', s.n_spectrum)
print('Total Number of Spectra:', s.n_spectra)
print("Front trim :", s.front_trim_amount)
print("End trim :", s.end_trim_amount)
print('Data Start Input:',s.data_start)
print('Data End Input:',s.data_end)           
print('Sample Size of training data:', s.samplesize )
print('Rows discarded:', s.n_discard_rows)
print('Resolution (1/cm) = ', s.resolution)

print('\n labels of molecules present \n', s.labels)
print('\n target indices (integers) of molecules present', s.targets)
print('\n frequencies present in the data \n ', s.frequencies)

## visualize the IR spectra

In [ ]:
def f_spectra(spectra_no):
    plt.plot(s.frequencies, s.spectra[spectra_no]); #reshape needed so you have 367 datapoints.
#     plt.ylim(-0.5, 5)
    plt.grid(True)
    plt.show()

interactive_plot = interactive(f_spectra, spectra_no=(0, s.spectra.shape[0]-1))
output = interactive_plot.children[-1]
interactive_plot

In [ ]:
s.make_dataframe(s.spectra)
spectraframe = s.spectraframe
spectraframe['mean_abs'] = spectraframe.drop(labels=['labels','targets'],axis = 1).mean(axis = 0)
spectraframe['norm_std_abs'] = spectraframe.drop(labels=['labels','targets'],axis = 1).std(axis = 0)/spectraframe.drop(labels=['labels','targets'],axis = 1).mean(axis = 0)
spectraframe['norm_max_abs'] = spectraframe.drop(labels=['labels','targets'],axis = 1).max(axis = 0)/spectraframe.drop(labels=['labels','targets'],axis = 1).mean(axis = 0)

# analytics for training and testing data

In [ ]:
X = s.spectra
y = s.targets
labels = s.labels
n_compounds = s.n_compounds
n_spectrum = s.n_spectrum
n_spectra = s.n_compounds*s.n_spectrum
samplesize = s.samplesize
wavenumbers = s.frequencies

In [ ]:
from sklearn.model_selection import train_test_split
TRAIN_SIZE=0.70
TEST_SIZE=1-TRAIN_SIZE

indices = np.arange(n_spectra)

train_X, test_X, train_y, test_y, train_indices, test_indices = train_test_split(X, y, indices, train_size=TRAIN_SIZE,
                                                   test_size=TEST_SIZE,
                                                   random_state=123,
                                                   stratify=y
                                                   )

print("All:", np.bincount(y) / float(len(y))*100  )
print("Training:", np.bincount(train_y) / float(len(train_y))*100  )
print("Testing:", np.bincount(test_y) / float(len(test_y))*100  )



In [ ]:
train_sf = spectraframe.iloc[train_indices].sort_index()
test_sf = spectraframe.iloc[test_indices].sort_index()




In [ ]:
plot_compound_counts(train_sf, 'Counts (Training Spectra)', color = 'red' )
plot_compound_counts(test_sf, 'Counts (Testing Spectra)', color = 'blue' )
plot_dataset_property(train_sf, "norm_max_abs", title = 'Normalized maximum absorbance (training Spectra)',  ylabel = 'Normalized maximum absorbance', color = 'red')
plot_dataset_property(test_sf, "norm_max_abs", title = 'Normalized maximum absorbance (testing Spectra)',  ylabel = 'Normalized maximum absorbance', color = 'blue')
# plot_dataset_property(train_sf, "mean_abs", title = 'Mean Absorbance (Training Spectra)',  ylabel = 'Mean Absorbance')
# plot_dataset_property(test_sf, "mean_abs", title = 'Mean Absorbance (Test Spectra)',  ylabel = 'Mean Absorbance')
plot_dataset_property(train_sf, "norm_std_abs", title = 'Normalized standard deviation absorbance (training Spectra)',  ylabel = 'Normalized standard deviation absorbance', color = 'red')
plot_dataset_property(test_sf, "norm_std_abs", title = 'Normalized standard deviation absorbance (testing Spectra)',  ylabel = 'Normalized standard deviation absorbance', color = 'blue')


# noisy simulated validation data


In [ ]:
s.add_sinusoidal_noise()
s.make_dataframe(s.val_sim_spectra)
spectraframe = s.spectraframe
spectraframe['mean_abs'] = spectraframe.drop(labels=['labels','targets'],axis = 1).mean(axis = 0)
# spectraframe['std_abs'] = spectraframe.drop(labels=['labels','targets'],axis = 1).std(axis = 0)
# spectraframe['max_abs'] = spectraframe.drop(labels=['labels','targets'],axis = 1).max(axis = 0)
spectraframe['norm_std_abs'] = spectraframe.drop(labels=['labels','targets'],axis = 1).std(axis = 0)/spectraframe.drop(labels=['labels','targets'],axis = 1).mean(axis = 0)
spectraframe['norm_max_abs'] = spectraframe.drop(labels=['labels','targets'],axis = 1).max(axis = 0)/spectraframe.drop(labels=['labels','targets'],axis = 1).mean(axis = 0)


In [ ]:
spectraframe

In [ ]:
def f_spectra(spectra_no):
    
    plt.subplot(2,1,1)
    plt.plot(s.frequencies, s.spectra[spectra_no]); #reshape needed so you have 367 datapoints.
    plt.subplot(2,1,2)
    plt.plot(s.frequencies, s.val_sim_spectra[spectra_no]); #reshape needed so you have 367 datapoints.
#     plt.ylim(-0.5, 5)
    plt.grid(True)
    plt.show()

interactive_plot = interactive(f_spectra, spectra_no=(0, s.spectra.shape[0]-1))
output = interactive_plot.children[-1]
interactive_plot

In [ ]:
plot_compound_counts(spectraframe, 'Counts (Validation Spectra)', color = 'green' )

plot_dataset_property(spectraframe, "norm_max_abs", title = 'Normalized maximum absorbance (validation Spectra)',  ylabel = 'Normalized maximum absorbance', color = 'green')
plot_dataset_property(spectraframe, "norm_std_abs", title = 'Normalized standard deviation absorbance (validation Spectra)',  ylabel = 'Normalized standard deviation absorbance', color = 'green')
# plot_dataset_property(spectraframe, "mean_abs", title = 'Mean absorbance (validation Spectra)',  ylabel = 'Mean absorbance', color = 'green')



## load IR data for pressure cross-validation, view attributes and visualize data

In [ ]:
s = IR_data(data_start = 400, data_end = 4000, resolution=1, verbosity = True, cv_type = 'pressure')
s.load_IR_data()

In [ ]:
print('Number of Compounds:', s.n_compounds)
print('Number of Spectrum:', s.n_spectrum)
print('Total Number of Spectra:', s.n_spectra)
print("Front trim :", s.front_trim_amount)
print("End trim :", s.end_trim_amount)
print('Data Start Input:',s.data_start)
print('Data End Input:',s.data_end)           
print('Sample Size of training data:', s.samplesize )
print('Rows discarded:', s.n_discard_rows)
print('Resolution (1/cm) = ', s.resolution)

print('\n labels of molecules present \n', s.labels)
print('\n target indices (integers) of molecules present', s.targets)
print('\n frequencies present in the data \n ', s.frequencies)

In [ ]:
def f_spectra(spectra_no):
    plt.plot(s.frequencies, s.spectra[spectra_no]); #reshape needed so you have 367 datapoints.
#     plt.ylim(-0.5, 5)
    plt.grid(True)
    plt.show()

interactive_plot = interactive(f_spectra, spectra_no=(0, s.spectra.shape[0]-1))
output = interactive_plot.children[-1]
interactive_plot

In [ ]:
s.make_dataframe(s.spectra)
spectraframe = s.spectraframe
spectraframe['mean_abs'] = spectraframe.drop(labels=['labels','targets'],axis = 1).mean(axis = 0)
spectraframe['norm_std_abs'] = spectraframe.drop(labels=['labels','targets'],axis = 1).std(axis = 0)/spectraframe.drop(labels=['labels','targets'],axis = 1).mean(axis = 0)
spectraframe['norm_max_abs'] = spectraframe.drop(labels=['labels','targets'],axis = 1).max(axis = 0)/spectraframe.drop(labels=['labels','targets'],axis = 1).mean(axis = 0)
plot_compound_counts(spectraframe, 'Counts (cross-validation on pressure)', color = 'red' )

plot_dataset_property(spectraframe, "norm_max_abs", title = 'Normalized maximum absorbance (cross-validation on pressure)',  ylabel = 'Normalized maximum absorbance', color = 'red')

# plot_dataset_property(spectraframe, "mean_abs", title = 'Mean Absorbance (Training Spectra)',  ylabel = 'Mean Absorbance')
plot_dataset_property(spectraframe, "norm_std_abs", title = 'Normalized standard deviation absorbance (cross-validation on pressure)',  ylabel = 'Normalized standard deviation absorbance', color = 'red')

## load IR data for concentration cross-validation, view attributes and visualize data

In [ ]:
s = IR_data(data_start = 400, data_end = 4000, resolution=1, verbosity = True, cv_type = 'concentration')
s.load_IR_data()

In [ ]:
print('Number of Compounds:', s.n_compounds)
print('Number of Spectrum:', s.n_spectrum)
print('Total Number of Spectra:', s.n_spectra)
print("Front trim :", s.front_trim_amount)
print("End trim :", s.end_trim_amount)
print('Data Start Input:',s.data_start)
print('Data End Input:',s.data_end)           
print('Sample Size of training data:', s.samplesize )
print('Rows discarded:', s.n_discard_rows)
print('Resolution (1/cm) = ', s.resolution)

print('\n labels of molecules present \n', s.labels)
print('\n target indices (integers) of molecules present', s.targets)
print('\n frequencies present in the data \n ', s.frequencies)

In [ ]:
s.make_dataframe(s.spectra)
spectraframe = s.spectraframe
spectraframe['mean_abs'] = spectraframe.drop(labels=['labels','targets'],axis = 1).mean(axis = 0)
spectraframe['norm_std_abs'] = spectraframe.drop(labels=['labels','targets'],axis = 1).std(axis = 0)/spectraframe.drop(labels=['labels','targets'],axis = 1).mean(axis = 0)
spectraframe['norm_max_abs'] = spectraframe.drop(labels=['labels','targets'],axis = 1).max(axis = 0)/spectraframe.drop(labels=['labels','targets'],axis = 1).mean(axis = 0)
plot_compound_counts(spectraframe, 'Counts (cross-validation on concentration)', color = 'red' )

plot_dataset_property(spectraframe, "norm_max_abs", title = 'Normalized maximum absorbance (cross-validation on concentration)',  ylabel = 'Normalized maximum absorbance', color = 'red')

# plot_dataset_property(spectraframe, "mean_abs", title = 'Mean Absorbance (Training Spectra)',  ylabel = 'Mean Absorbance')
plot_dataset_property(spectraframe, "norm_std_abs", title = 'Normalized standard deviation absorbance (cross-validation on concentration)',  ylabel = 'Normalized standard deviation absorbance', color = 'red')

# experimental data

In [ ]:
path_exp = "../../data/IR_Experimental_Data/"

freq_H2O, abs_H2O = load_exp_spectra(path_exp, 'H2O-4-NIST.xlsx')
freq_CO2, abs_CO2 = load_exp_spectra(path_exp, 'CO2-4-NIST.xlsx')
freq_CO, abs_CO = load_exp_spectra(path_exp, 'CO-4-NIST.xlsx')
freq_N2O, abs_N2O = load_exp_spectra(path_exp, 'B_N2O-1-NIST.xlsx')

freq_CH4, abs_CH4 = load_exp_spectra(path_exp, 'CH4-1.xlsx')
freq_NO, abs_NO = load_exp_spectra(path_exp, 'B_NO-1-NIST.xlsx')
freq_NH3, abs_NH3 = load_exp_spectra(path_exp, 'B_NH3-4.xlsx')
freq_H2CO, abs_H2CO = load_exp_spectra(path_exp, 'H2COUnknown.xlsx')

freq_CH3Cl, abs_CH3Cl = load_exp_spectra(path_exp, 'CH3CL.xlsx')
freq_HBr, abs_HBr = load_exp_spectra(path_exp, 'HBr.xlsx')
freq_OCS, abs_OCS = load_exp_spectra(path_exp, 'OCS.xlsx')
freq_C2H2, abs_C2H2 = load_exp_spectra(path_exp, 'C2H2.xlsx')

freq_C2H4, abs_C2H4 = load_exp_spectra(path_exp, 'C2H4.xlsx')
freq_C2H6, abs_C2H6 = load_exp_spectra(path_exp, 'C2H6_upto_2400.xlsx')
freq_SO2, abs_SO2 = load_exp_spectra(path_exp, 'SO2.xlsx')
freq_O3, abs_O3 = load_exp_spectra(path_exp, 'O3-4.xlsx')

freq_HCl, abs_HCl = load_exp_spectra(path_exp, 'HCl_25T_Full_Shift.xlsx')
freq_H2S, abs_H2S = load_exp_spectra(path_exp, 'H2S.xlsx')
freq_CH3Br, abs_CH3Br = load_exp_spectra(path_exp, 'CH3Br_Short.xlsx')
freq_HC3N, abs_HC3N = load_exp_spectra(path_exp, 'HC3N.xlsx')



In [ ]:
exp_means = [np.mean(abs_H2O),np.mean(abs_CO2),np.mean(abs_CO),np.mean(abs_N2O),
np.mean(abs_CH4),np.mean(abs_NO),np.mean(abs_NH3),np.mean(abs_H2CO),
np.mean(abs_CH3Cl),np.mean(abs_HBr),np.mean(abs_OCS),np.mean(abs_C2H2),
np.mean(abs_C2H4),np.mean(abs_C2H6),np.mean(abs_SO2),np.mean(abs_O3),
np.mean(abs_HCl),np.mean(abs_H2S),np.mean(abs_CH3Br),np.mean(abs_HC3N)]





In [ ]:
exp_maxs = [np.max(abs_H2O),np.max(abs_CO2),np.max(abs_CO),np.max(abs_N2O),
np.max(abs_CH4),np.max(abs_NO),np.max(abs_NH3),np.max(abs_H2CO),
np.max(abs_CH3Cl),np.max(abs_HBr),np.max(abs_OCS),np.max(abs_C2H2),
np.max(abs_C2H4),np.max(abs_C2H6),np.max(abs_SO2),np.max(abs_O3),
np.max(abs_HCl),np.max(abs_H2S),np.max(abs_CH3Br),np.max(abs_HC3N)]


In [ ]:
exp_stds = [np.std(abs_H2O),np.std(abs_CO2),np.std(abs_CO),np.std(abs_N2O),
np.std(abs_CH4),np.std(abs_NO),np.std(abs_NH3),np.std(abs_H2CO),
np.std(abs_CH3Cl),np.std(abs_HBr),np.std(abs_OCS),np.std(abs_C2H2),
np.std(abs_C2H4),np.std(abs_C2H6),np.std(abs_SO2),np.std(abs_O3),
np.std(abs_HCl),np.std(abs_H2S),np.std(abs_CH3Br),np.std(abs_HC3N)]





In [ ]:
exp_labels = ['H2O','CO2','CO','N2O',
'CH4','NO','NH3','H2CO',
'CH3Cl','HBr','OCS','C2H2',
'C2H4','C2H6','SO2','O3',
'HCl','H2S','CH3Br','HC3N']

In [ ]:
exp_norm_max_abs =  np.array(exp_maxs)/np.array(exp_means)

In [ ]:
exp_norm_std_abs =  np.array(exp_stds)/np.array(exp_means)

In [ ]:
exp_df = pd.DataFrame()
exp_df['mean_abs'] = exp_means
exp_df['norm_max_abs'] = exp_norm_max_abs
exp_df['norm_std_abs'] = exp_norm_std_abs
exp_df['labels'] = exp_labels


In [ ]:
exp_df

In [ ]:
plot_compound_counts(exp_df, title = 'counts (experimental spectra)', color = 'blue', save_to_file = True)

In [ ]:
plot_dataset_property(exp_df, 'norm_max_abs', title = 'Normalized maximum absorbance (experimental spectra)',  ylabel = 'Normalized Maximum Absorbance', color = 'red', save_to_file = True)    
plot_dataset_property(exp_df, 'norm_std_abs', title = 'Normalized standard deviation (experimental spectra)',  ylabel = 'Normalized Standard Deviation Absorbance', color = 'red', save_to_file = True)    

## notebook ends